In [ ]:
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
%matplotlib inline

folder=tb.latest_data('164501', folder ='Y:\data')

analyze_lt3 = False
# lt3 vs lt4 PSB
name = 'adwindata'
a = pqsequence.TailAnalysis(folder, hdf5_mode='r')#, pq_folder='bs_remote')

a.get_sweep_pts()
#a.sweep_pts = a.g.attrs['aom_amplitude']
a.get_sweep_idxs(noof_syncs_per_sweep_pt=250)
a.get_readout_results(name)
if analyze_lt3:
    a.get_tail_vs_sweep(0,2720,100, pq_binsize_ns=1, hist_binsize_ns=1.,verbose=True)
else:
    a.get_tail_vs_sweep(1,2790,100, pq_binsize_ns=1, hist_binsize_ns=1.,verbose=True)
    


In [ ]:
if analyze_lt3:
    pulse_start = 2757
else:
    pulse_start = 2800
fit_length = 35

x=a.tail_hist_b[:-1] 
y= a.tail_hist_h.squeeze()

arg_start = np.argmin(abs(x-pulse_start))
arg_end = np.argmin(abs(x-pulse_start-fit_length))

offset =  x[arg_start] 
x_for_fit=x[arg_start:arg_end] -  offset
y_for_fit=y[arg_start:arg_end]

### create a plot
fig = plt.figure(figsize =(7,7))
ax1 = fig.add_subplot(211)
if analyze_lt3:
    plt.title(a.default_plot_title + ', LT3 analysis')
else:
    plt.title(a.default_plot_title + ', LT4 analysis')
   
plt.xlabel('Time (ns)')
plt.ylabel('Counts')

ax2 = fig.add_subplot(212)
plt.xlabel('Time (ns)')
plt.ylabel('Counts')

## plot data
ax1.plot(x,y,linewidth = 2)
ax2.plot(x_for_fit,y_for_fit,'b.',markersize =8)

def fit_rabi(g_f, g_A, g_x0, g_phi,g_t1,g_t2):
    fitfunc_str = 'A *(exp(-x/t1) cos(2pi * (f*x + phi/360) ) + 1)*exp(-x/t2)'

    f = fit.Parameter(g_f, 'f')
    A = fit.Parameter(g_A, 'A')
    x0 = fit.Parameter(g_x0, 'x0')
    phi = fit.Parameter(g_phi, 'phi')
    t1   = fit.Parameter(g_t1, 't1')
    t2   = fit.Parameter(g_t2, 't2')
    p0 = [f, A,x0,phi,t1,t2]

    def fitfunc(x):
        return A()*(np.exp(-(x-x0())/t1()) * np.cos(2*np.pi*( f()*(x-x0()) + phi()/360.))+1)*np.exp(-(x-x0())/t2())

    return p0, fitfunc, fitfunc_str


def fit_rabi_2(g_f, g_A,g_x0, g_phi,g_t1):
    fitfunc_str = 'A*(exp(-x/t1) cos(2pi * (f*x + phi/360) ) +1)'

    f = fit.Parameter(g_f, 'f')
    A = fit.Parameter(g_A, 'A')
    x0 = fit.Parameter(g_x0, 'x0')
    phi = fit.Parameter(g_phi, 'phi')
    t1   = fit.Parameter(g_t1, 't1')
    p0 = [f, A,x0,phi,t1]

    def fitfunc(x):
        return (A()*(np.exp(-(x-x0())/t1()) * np.cos(2*np.pi*( f()*(x-x0()) + phi()/360.))+1))

    return p0, fitfunc, fitfunc_str

#p0, fitfunc, fitfunc_str = common.fit_decaying_cos(0.15, 1e4, 1e4, 170,20)
# 'A *exp(-x/t) cos(2pi * (f*x + phi/360) ) + a'
# g_f, g_a, g_A, g_phi,g_t

p0, fitfunc, fitfunc_str = fit_rabi(0.03, 1.1e4,0, 180,15,200)

#p0, fitfunc, fitfunc_str = fit_rabi_2(0.03, 0.7e4,0,180,16)

show_guess = True
if show_guess:
    # print decay
    ax1.plot(np.linspace(x_for_fit[0]+offset,x_for_fit[-1]+offset,201), fitfunc(np.linspace(x_for_fit[0],x_for_fit[-1],201)), ':', lw=2)
    ax2.plot(np.linspace(x_for_fit[0],x_for_fit[-1],201), fitfunc(np.linspace(x_for_fit[0],x_for_fit[-1],201)), ':', lw=2)

fit_result = fit.fit1d(x_for_fit, y_for_fit, None, p0=p0, fitfunc=fitfunc, 
        ret = True,fixed=[3,4],do_print=True)

if isinstance(fit_result, int):
    print "Fit failed!"
else: 
    ax1.plot(np.linspace(x_for_fit[0]+offset,x_for_fit[-1]+offset,201), fit_result['fitfunc'](np.linspace(x_for_fit[0],x_for_fit[-1],201)),'r')
    ax2.plot(np.linspace(x_for_fit[0],x_for_fit[-1],201),fit_result['fitfunc'](np.linspace(x_for_fit[0],x_for_fit[-1],201)),'r')

In [ ]:
1-16/62.

In [ ]:
lt3_rabi_freq = [0.125,0.110,0.033,0.188]
lt3_tau = [16.8,17.6,20.5,13.8]

lt4_rabi_freq = [0.167,0.159,0.040,0.218,0.057,0.123,0.298,0.266,0.087]
lt4_tau = [13.7,15.7,21.8,10.5,19.5,18.6,11.8,12.5,18.2]


### create a plot
fig = plt.figure(figsize =(7,10))
ax1 = fig.add_subplot(211)

plt.xlabel('Rabi frequency (GHz)')
plt.ylabel('Decay constant (ns)')
lt3_plot = plt.scatter(lt3_rabi_freq,lt3_tau)
lt4_plot = plt.scatter(lt4_rabi_freq,lt4_tau,marker='x')
plt.legend([lt3_plot,lt4_plot],['LT3','LT4'])
plt.show()

In [ ]:
#### just to get a clearer picture here is the textbook solution for a coherence decay limited by spont. emission.

